In [2]:
from glob import glob
from itertools import chain
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import requests
from requests.exceptions import JSONDecodeError
import json
import seaborn as sns
import yfinance as yf
from fredapi import Fred
from numba import float64, guvectorize, int64, njit, vectorize
from pandas.tseries.offsets import BMonthEnd
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
#pd.set_option('display.max_columns', None)

In [3]:
def group_by_b_month_end(dt):
    end_date = dt + BMonthEnd(0)
    return end_date

In [4]:
def read_msci_data(filename):
    df = pd.read_excel(filename, skiprows=6, skipfooter=19)
    df.columns = ['date', 'price']
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index('date')
    df = df.replace(',','', regex=True)
    df['price'] = df['price'].astype(float)
    return df

In [5]:
msci_world = read_msci_data('data/MSCI World USD Gross.xls')

In [6]:
def extract_financialtimes_data(filepaths):
    dfs = [pd.read_html(filepath)[2].iloc[::-1] for filepath in filepaths]
    df = pd.concat(dfs, ignore_index=True)
    df['Date'] = pd.to_datetime(df['Date'].apply(lambda x: ''.join(x.rsplit(',', maxsplit=2)[-2:])[1:]))
    df = df[df['Date'].isin(pd.date_range(df['Date'].iloc[0], df['Date'].iloc[-1], freq='BM'))]
    df = df.reset_index(drop=True)
    df = df[['Date', 'Close']]
    df.columns = ['date', 'price']
    df = df.set_index('date')
    return df
    

In [7]:
sti = extract_financialtimes_data(glob('data/STI Data/*/*.htm'))

In [50]:
def download_fed_funds_rate():
    fred = Fred()
    fed_funds_rate = fred.get_series('DFF').rename('ffr').rename_axis('date')
    fed_funds_rate.to_csv('data/fed_funds_rate.csv')
    return fed_funds_rate

In [55]:
def load_fed_funds_rate():
    try:
        fed_funds_rate = pd.read_csv('data/fed_funds_rate.csv', parse_dates=['date'])
        if pd.to_datetime(fed_funds_rate['date']).iloc[-1] < pd.to_datetime('today') + BMonthEnd(-1, 'D'):
            raise FileNotFoundError
        fed_funds_rate = fed_funds_rate.set_index('date')['ffr']
    
    except FileNotFoundError:
        fed_funds_rate = download_fed_funds_rate()
    
    fed_funds_rate_1m = fed_funds_rate.div(36000).add(1).groupby(group_by_b_month_end).prod().pow(12).sub(1).mul(100)
    
    return fed_funds_rate, fed_funds_rate_1m

In [56]:
fed_funds_rate, fed_funds_rate_1m = load_fed_funds_rate()

In [11]:
sp500 = yf.download('^SP500TR')['Adj Close']

[*********************100%%**********************]  1 of 1 completed


In [12]:
sp500 = sp500.groupby(group_by_b_month_end).last()

In [13]:
def read_shiller_sp500_data(net=False):
    df = pd.read_excel('data/ie_data.xls', 'Data', skiprows=range(7), skipfooter=1).drop(['Unnamed: 13','Unnamed: 15'], axis=1)
    df.index = pd.to_datetime(df['Date'].astype(str).str.split('.').apply(lambda x: '-'.join(x)).str.ljust(7, '0')) + BMonthEnd(0)
    shiller_sp500 = df['P'].add(df['D'].ffill().div(12).mul(0.7 if net else 1)).div(df['P'].shift(1)).fillna(1).cumprod()
    shiller_sp500 = shiller_sp500.rename('shiller_sp500')
    return shiller_sp500

In [14]:
shiller_sp500 = read_shiller_sp500_data()

In [15]:
def download_usdsgd():
    usd_sgd_response = requests.get('https://eservices.mas.gov.sg/api/action/datastore/search.json',
                   params={'resource_id': '10eafb90-11a2-4fbd-b7a7-ac15a42d60b6',
                           'between[end_of_month]': f'1969-12,{pd.to_datetime("today").strftime("%Y-%m")}',
                           'fields': 'end_of_month,usd_sgd'
                           }
                   ).json()
    usdsgd = pd.DataFrame(usd_sgd_response['result']['records'])[['end_of_month', 'usd_sgd']]
    usdsgd['end_of_month'] = pd.to_datetime(usdsgd['end_of_month']) + BMonthEnd()
    return usdsgd

In [16]:
usdsgd = download_usdsgd()

In [17]:
def download_sgd_interest_rates():
    offset = 0
    dfs = []
    with requests.Session() as session:
        while True:
            sgd_interest_rates_response = session.get('https://eservices.mas.gov.sg/api/action/datastore/search.json',
                        params={'resource_id': '9a0bf149-308c-4bd2-832d-76c8e6cb47ed',
                                'between[end_of_day]': f'1987-07-01,{pd.to_datetime("today").strftime("%Y-%m-%d")}',
                                'offset': f'{offset}',
                                'fields': 'end_of_day,interbank_overnight,sora'
                                }
                        ).json()
            df = pd.DataFrame(sgd_interest_rates_response['result']['records'])[['end_of_day', 'interbank_overnight', 'sora']]
            offset += 100
            dfs.append(df)
            if len(df) < 100:
                break
    sgd_interest_rates = pd.concat(dfs)
    sgd_interest_rates['interbank_overnight'] = sgd_interest_rates['interbank_overnight'].astype(float)
    sgd_interest_rates['end_of_day'] = pd.to_datetime(sgd_interest_rates['end_of_day'])
    sgd_interest_rates = sgd_interest_rates.dropna(how='all', subset=['interbank_overnight', 'sora'])
    sgd_interest_rates = sgd_interest_rates.drop_duplicates().drop_duplicates(subset=['end_of_day', 'interbank_overnight']).drop_duplicates(subset=['end_of_day', 'sora'])
    sgd_interest_rates = sgd_interest_rates.reset_index(drop=True)
    sgd_interest_rates = sgd_interest_rates.set_index('end_of_day')
    return sgd_interest_rates

In [18]:
def load_sgd_interest_rates():
    try:
        sgd_interest_rates = pd.read_csv('data/sgd_interest_rates.csv', parse_dates=['end_of_day'])
        if pd.to_datetime(sgd_interest_rates['end_of_day']).iloc[-1] < pd.to_datetime('today') + BMonthEnd(-1):
            raise FileNotFoundError
        sgd_interest_rates = sgd_interest_rates.set_index('end_of_day')
        
    except FileNotFoundError:
        sgd_interest_rates = download_sgd_interest_rates()
        sgd_interest_rates.to_csv('data/sgd_interest_rates.csv')
        
    sgd_interest_rates_1m = sgd_interest_rates.resample('D').ffill().div(36000).add(1).groupby(group_by_b_month_end).prod().pow(12).sub(1).mul(100).replace(0, np.nan)
    sgd_interest_rates_1m.loc['2014-01-31', 'interbank_overnight'] = np.nan
    sgd_interest_rates_1m['sgd_ir_1m'] = sgd_interest_rates_1m['interbank_overnight'].fillna(sgd_interest_rates['sora'])
    return sgd_interest_rates, sgd_interest_rates_1m

In [19]:
sgd_interest_rates, sgd_interest_rates_1m = load_sgd_interest_rates()

In [20]:
def download_sg_cpi():
    try:
        sg_cpi_response = requests.get('https://tablebuilder.singstat.gov.sg/api/table/tabledata/M212882')
        sg_cpi = pd.DataFrame(sg_cpi_response.json()['Data']['row'][0]['columns'])
        sg_cpi.columns = ['date', 'sg_cpi']
        sg_cpi['date'] = pd.to_datetime(sg_cpi['date']) + BMonthEnd()
        sg_cpi = sg_cpi.set_index('date')
    except JSONDecodeError:
        sg_cpi = pd.read_csv('data/sg_cpi.csv', index_col='date')
    return sg_cpi

In [21]:
def load_sg_cpi():
    try:
        sg_cpi = pd.read_csv('data/sg_cpi.csv', parse_dates=['date'])
        if pd.to_datetime(sg_cpi['date']).iloc[-1] < pd.to_datetime('today') + BMonthEnd(-1, 'D'):
            raise FileNotFoundError
        sg_cpi = sg_cpi.set_index('date')
        return sg_cpi
    except FileNotFoundError:
        sg_cpi = download_sg_cpi()
        sg_cpi.to_csv('data/sg_cpi.csv')
        return sg_cpi

In [22]:
sg_cpi = load_sg_cpi()

In [47]:
def download_us_cpi():
    with requests.Session() as session:
        dfs = [
            pd.DataFrame(
                session.post(
                    'https://api.bls.gov/publicAPI/v2/timeseries/data/',
                    json={'seriesid': ['CUSR0000SA0'],
                        'startyear': f'{year}',
                        'endyear': f'{year+9}',
                        'catalog': 'true',
                        'registrationkey': os.environ['BLS_API_KEY']
                        },
                    headers={'Content-Type': 'application/json'}
                ).json()['Results']['series'][0]['data']
            ).iloc[::-1]
        for year in range(1947, 2023, 10)
        ]
    us_cpi = pd.concat(dfs).reset_index(drop=True)
    us_cpi['month'] = us_cpi['period'].str[-2:]
    us_cpi['date'] = pd.to_datetime(us_cpi['year'] + '-' + us_cpi['month']) + BMonthEnd()
    us_cpi['value'] = us_cpi['value'].astype(float)
    us_cpi = us_cpi[['date', 'value']]
    us_cpi.columns = ['date', 'us_cpi']
    us_cpi = us_cpi.set_index('date')
    return us_cpi

In [48]:
def load_us_cpi():
    try:
        us_cpi = pd.read_csv('data/us_cpi.csv', parse_dates=['date'])
        if pd.to_datetime(us_cpi['date']).iloc[-1] < pd.to_datetime('today') + BMonthEnd(-1, 'D'):
            raise FileNotFoundError
        us_cpi = us_cpi.set_index('date')
        return us_cpi
    except FileNotFoundError:
        us_cpi = download_us_cpi()
        us_cpi.to_csv('data/us_cpi.csv')
        return us_cpi

In [49]:
us_cpi = load_us_cpi()

In [26]:
msci_world = msci_world.merge(fed_funds_rate_1m, left_index=True, right_index=True, how='left')

In [27]:
msci_world = msci_world.merge(sgd_interest_rates_1m['sgd_ir_1m'], left_index=True, right_index=True, how='left')

In [28]:
periods = ['1m', '3m', '6m', '1y', '2y', '3y', '5y', '10y', '15y', '20y', '25y', '30y']
durations = [1, 3, 6, 12, 24, 36, 60, 120, 180, 240, 300, 360]

In [29]:
@njit
def calculate_return(ending_index, dca_length, monthly_returns, investment_horizon=None):
    if investment_horizon is None:
        investment_horizon = dca_length
    elif investment_horizon < dca_length:
        raise ValueError('Investment horizon must be greater than or equal to DCA length')
    if ending_index < dca_length:
        return np.nan
    share_value = 0
    cash = 1
    for i in range(ending_index - investment_horizon, ending_index - investment_horizon + dca_length):
        cash -= 1/dca_length
        share_value += 1/dca_length
        share_value *= 1 + monthly_returns[i+1]
    for i in range(ending_index - investment_horizon + dca_length, ending_index):
        share_value *= 1 + monthly_returns[i+1]
    return share_value - 1

@guvectorize([(int64, float64[:], int64, float64[:])], '(),(n),()->(n)', target='parallel', nopython=True)
def calculate_return_vector(dca_length, monthly_returns, investment_horizon, res=np.array([])):
    if investment_horizon < dca_length:
        raise ValueError('Investment horizon must be greater than or equal to DCA length')
    for i in range(len(monthly_returns)):
        if i < investment_horizon:
            res[i] = np.nan
        share_value = 0
        cash = 1
        for j in range(i - investment_horizon, i - investment_horizon + dca_length):
            cash -= 1/dca_length
            share_value += 1/dca_length
            share_value *= 1 + monthly_returns[j+1]
        for j in range(i - investment_horizon + dca_length, i):
            share_value *= 1 + monthly_returns[j+1]
        res[i] = share_value - 1

@guvectorize([(float64, float64, float64, float64, int64, int64, int64, float64[:], float64[:], float64[:])], '(),(),(),(),(),(),(),(n),(n)->(n)', target='parallel', nopython=True)
def calculate_lumpsum_return_with_fees_and_interest_vector(variable_transaction_fees, fixed_transaction_fees, annualised_holding_fees, total_investment, dca_length, dca_interval, investment_horizon, monthly_returns, interest_rates, res=np.array([])):
    if investment_horizon < dca_length:
        raise ValueError('Investment horizon must be greater than or equal to DCA length')
    if fixed_transaction_fees >= total_investment / dca_length * dca_interval:
        raise ValueError('Fixed fees must be less than the amount invested in each DCA')
    for i in range(len(monthly_returns)):
        if i < investment_horizon:
            res[i] = np.nan
        share_value = 0
        cash = total_investment
        monthly_amount = total_investment / dca_length
        for index, j in enumerate(range(i - investment_horizon, i - investment_horizon + dca_length)):
            if index % dca_interval == 0:
                dca_amount = cash - (dca_length - index - 1) * monthly_amount
                share_value += dca_amount * (1 - variable_transaction_fees) - fixed_transaction_fees
                cash = (dca_length - index - 1) * monthly_amount
            share_value *= ((1 + monthly_returns[j+1]) ** 12 - annualised_holding_fees) ** (1/12)
            cash *= (1 + interest_rates[j+1] / 100) ** (1/12)
        share_value += cash
        cash = 0
        for j in range(i - investment_horizon + dca_length, i):
            share_value *= 1 + monthly_returns[j+1]
        res[i] = (share_value - total_investment) / total_investment

@guvectorize([(float64, float64, float64, float64, int64, int64, float64[:], float64[:], float64[:])], '(),(),(),(),(),(),(n),(n)->(n)', target='parallel', nopython=True)
def calculate_dca_return_with_fees_and_interest_vector(variable_transaction_fees, fixed_transaction_fees, annualised_holding_fees, monthly_amount, dca_length, dca_interval, monthly_returns, interest_rates, res=np.array([])):
    total_investment = monthly_amount * dca_length
    dca_amount = monthly_amount * dca_interval
    if fixed_transaction_fees >= dca_amount:
        raise ValueError('Fixed fees must be less than the amount invested in each DCA')
    for i in range(len(monthly_returns)):
        if i < dca_length:
            res[i] = np.nan
        share_value = 0
        funds_to_invest = 0
        for index, j in enumerate(range(i - dca_length, i)):
            funds_to_invest += monthly_amount
            if (index + 1) % dca_interval == 0:
                share_value += funds_to_invest * (1 - variable_transaction_fees) - fixed_transaction_fees
                funds_to_invest = 0
            share_value *= ((1 + monthly_returns[j+1]) ** 12 - annualised_holding_fees) ** (1/12)
            funds_to_invest *= (1 + interest_rates[j+1] / 100) ** (1/12)
        res[i] = (share_value + funds_to_invest - total_investment) / total_investment

In [30]:
def add_return_columns(df):
    for period, duration in zip(periods, durations):
        df[f'{period}_cumulative'] = df['price'].pct_change(periods=duration)
    for period, duration in zip(periods, durations):
        df[f'{period}_annualized'] = (1 + df[f'{period}_cumulative'])**(12/duration) - 1
    for period, duration in zip(periods, durations):
        df[f'{period}_dca_cumulative'] = calculate_return_vector(duration, df['1m_cumulative'].values, duration)
    for period, duration in zip(periods, durations):
        df[f'{period}_dca_annualized'] = (1 + df[f'{period}_dca_cumulative'])**(12/duration) - 1
    for period, duration in zip(periods, durations):
        df[f'{period}_cumulative_difference'] = df[f'{period}_cumulative'] - df[f'{period}_dca_cumulative']
    for period, duration in zip(periods, durations):
        df[f'{period}_difference_in_annualized'] = df[f'{period}_annualized'] - df[f'{period}_dca_annualized']

In [31]:
add_return_columns(msci_world)

In [32]:
add_return_columns(sti)

C:\Users\leowy\AppData\Local\Temp\ipykernel_23584\1388376671.py:7: RuntimeWarning: overflow encountered in calculate_return_vector
  df[f'{period}_dca_cumulative'] = calculate_return_vector(duration, df['1m_cumulative'].values, duration)
C:\Users\leowy\AppData\Local\Temp\ipykernel_23584\1388376671.py:7: RuntimeWarning: overflow encountered in calculate_return_vector
  df[f'{period}_dca_cumulative'] = calculate_return_vector(duration, df['1m_cumulative'].values, duration)
C:\Users\leowy\AppData\Local\Temp\ipykernel_23584\1388376671.py:7: RuntimeWarning: overflow encountered in calculate_return_vector
  df[f'{period}_dca_cumulative'] = calculate_return_vector(duration, df['1m_cumulative'].values, duration)


In [33]:
msci_world.head(10)

,price,ffr,sgd_ir_1m,1m_cumulative,3m_cumulative,6m_cumulative,1y_cumulative,2y_cumulative,3y_cumulative,5y_cumulative,...,6m_difference_in_annualized,1y_difference_in_annualized,2y_difference_in_annualized,3y_difference_in_annualized,5y_difference_in_annualized,10y_difference_in_annualized,15y_difference_in_annualized,20y_difference_in_annualized,25y_difference_in_annualized,30y_difference_in_annualized
date,,,,,,,,,,,,,,,,,,,,,
1969-12-31,100.000,10.391636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-01-30,94.528,9.376079,NaN,-0.054720,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-27,97.558,8.771530,NaN,0.032054,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,97.947,8.657911,NaN,0.003987,-0.020530,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,88.877,8.432125,NaN,-0.092601,-0.059781,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-29,83.357,7.988168,NaN,-0.062108,-0.145565,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-06-30,81.389,8.469048,NaN,-0.023609,-0.169051,-0.186110,NaN,NaN,NaN,NaN,...,-0.098038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-07-31,86.525,7.739472,NaN,0.063104,-0.026464,-0.084663,NaN,NaN,NaN,NaN,...,-0.083847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-08-31,89.464,7.074040,NaN,0.033967,0.073263,-0.082966,NaN,NaN,NaN,NaN,...,-0.173593,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
msci_world.describe()

,price,ffr,sgd_ir_1m,1m_cumulative,3m_cumulative,6m_cumulative,1y_cumulative,2y_cumulative,3y_cumulative,5y_cumulative,...,6m_difference_in_annualized,1y_difference_in_annualized,2y_difference_in_annualized,3y_difference_in_annualized,5y_difference_in_annualized,10y_difference_in_annualized,15y_difference_in_annualized,20y_difference_in_annualized,25y_difference_in_annualized,30y_difference_in_annualized
count,644.000000,644.000000,430.000000,643.000000,641.000000,638.000000,632.000000,620.000000,608.000000,584.000000,...,638.000000,632.000000,620.000000,608.000000,584.000000,524.000000,464.000000,404.000000,344.000000,284.000000
mean,3048.848497,5.193560,1.713778,0.008647,0.026326,0.054115,0.111914,0.238008,0.372533,0.693159,...,0.055371,0.051956,0.050395,0.048760,0.046350,0.045198,0.043130,0.043411,0.043929,0.043796
std,3347.076797,4.284294,1.661664,0.043094,0.077306,0.115639,0.172692,0.275763,0.371367,0.622307,...,0.126908,0.096572,0.071563,0.057638,0.043038,0.026389,0.021232,0.016571,0.011823,0.008894
min,81.389000,0.048345,0.017001,-0.189341,-0.331171,-0.433765,-0.467637,-0.467822,-0.450059,-0.239983,...,-0.326142,-0.340453,-0.168074,-0.115756,-0.061955,-0.016487,-0.003166,0.004933,0.019341,0.027770
25%,310.390250,1.449395,0.305900,-0.015759,-0.011512,-0.011172,0.015095,0.090786,0.183184,0.259553,...,-0.018468,0.000298,0.013118,0.020883,0.024261,0.027818,0.028641,0.032993,0.034675,0.035284
50%,1967.580500,5.140584,1.255930,0.012223,0.029782,0.059273,0.128373,0.256179,0.357212,0.620335,...,0.049515,0.054249,0.057280,0.055361,0.049455,0.046465,0.040980,0.040246,0.040854,0.044731
75%,4387.986500,7.405355,2.755771,0.033657,0.073263,0.117279,0.207250,0.381873,0.557008,0.946393,...,0.121421,0.103904,0.089620,0.083094,0.069069,0.061483,0.060677,0.059328,0.055532,0.051887
max,14223.137000,23.069445,8.139264,0.147137,0.307832,0.472122,0.671366,1.405102,2.035154,3.456819,...,0.659631,0.398270,0.293252,0.209544,0.165537,0.110528,0.091993,0.077583,0.069472,0.061400


In [35]:
msci_world.loc[:, [*msci_world.loc[:,'1m_annualized':'30y_annualized'].columns, *msci_world.loc[:,'1m_dca_annualized':'30y_dca_annualized']]].describe()

,1m_annualized,3m_annualized,6m_annualized,1y_annualized,2y_annualized,3y_annualized,5y_annualized,10y_annualized,15y_annualized,20y_annualized,...,6m_dca_annualized,1y_dca_annualized,2y_dca_annualized,3y_dca_annualized,5y_dca_annualized,10y_dca_annualized,15y_dca_annualized,20y_dca_annualized,25y_dca_annualized,30y_dca_annualized
count,643.000000,641.000000,638.000000,632.000000,620.000000,608.000000,584.000000,524.000000,464.000000,404.000000,...,638.000000,632.000000,620.000000,608.000000,584.000000,524.000000,464.000000,404.000000,344.000000,284.000000
mean,0.239713,0.146542,0.124510,0.111914,0.105693,0.102477,0.100551,0.101980,0.101392,0.099773,...,0.069140,0.059959,0.055298,0.053716,0.054201,0.056782,0.058262,0.056361,0.055922,0.055366
std,0.617751,0.332298,0.241792,0.172692,0.124400,0.099172,0.075199,0.048631,0.042082,0.033344,...,0.146846,0.101057,0.072275,0.058295,0.044810,0.030178,0.027390,0.022853,0.019814,0.013793
min,-0.919451,-0.799894,-0.679378,-0.467637,-0.270495,-0.180708,-0.053404,-0.021078,0.032600,0.037583,...,-0.511205,-0.354103,-0.243771,-0.168423,-0.085712,-0.028866,-0.006182,0.009435,0.024426,0.036479
25%,-0.173545,-0.045260,-0.022219,0.015095,0.044407,0.057671,0.047233,0.070664,0.065619,0.070377,...,-0.006679,0.007636,0.021985,0.024255,0.028118,0.037078,0.038354,0.038155,0.039076,0.044372
50%,0.156953,0.124557,0.122058,0.128373,0.120794,0.107174,0.101339,0.096134,0.090462,0.096007,...,0.073276,0.072075,0.067613,0.060932,0.053362,0.054290,0.050689,0.047735,0.049338,0.048791
75%,0.487712,0.326860,0.248313,0.207250,0.175531,0.159036,0.142473,0.136246,0.146715,0.129590,...,0.155608,0.117077,0.095715,0.086757,0.077082,0.071645,0.081555,0.083684,0.070031,0.068743
max,4.192609,1.925556,1.167144,0.671366,0.550839,0.447861,0.348357,0.214958,0.187872,0.162350,...,0.581969,0.378061,0.294036,0.262130,0.204586,0.143609,0.115749,0.096955,0.097009,0.091788


In [36]:
go.Figure(
    data = [
        go.Box(
            x=msci_world[column],
            name=column,
            )
        for column in msci_world.loc[:,'1m_annualized':'30y_annualized'].columns
    ],
    layout = go.Layout(
        height=800,
        xaxis=dict(
            tickformat='.2%',
        )
    )
)

In [37]:
go.Figure(
    data = [
        go.Box(
            x=msci_world[column],
            name=column,
            )
        for column in chain.from_iterable(zip(msci_world.loc[:,'1m_annualized':'30y_annualized'].columns, msci_world.loc[:,'1m_dca_annualized':'30y_dca_annualized']))
    ],
    layout = go.Layout(
        height=800,
        xaxis=dict(
            tickformat='.2%',
        )
    )
)

In [38]:
go.Figure(
    [
        go.Scatter(
            x=msci_world.index,
            y=msci_world[column],
            name=column,
            mode='lines'
            )
        for column in ['5y_annualized', '5y_dca_annualized']
    ],
    layout = go.Layout(
        yaxis=dict(
            tickformat='.0%',
        )
    )
)

In [39]:
go.Figure(
    [
        go.Scatter(
            x=msci_world.index,
            y=msci_world[column],
            name=column,
            mode='lines'
            )
        for column in ['10y_annualized', '10y_dca_annualized']
    ],
    layout = go.Layout(
        yaxis=dict(
            tickformat='.0%',
        )
    )
)

In [40]:
go.Figure(
    [
        go.Scatter(
            x=msci_world.index,
            y=msci_world[column],
            name=column,
            mode='lines'
            )
        for column in ['20y_annualized', '20y_dca_annualized']
    ],
    layout = go.Layout(
        yaxis=dict(
            tickformat='.0%',
        )
    )
)

In [41]:
go.Figure(
    [
        go.Box(
            x=msci_world[column],
            name=column,
            opacity=0.75
            )
        for column in msci_world.loc[:, '1m_difference_in_annualized':'30y_difference_in_annualized'].columns
    ],
    layout = go.Layout(
        xaxis=dict(
            tickformat='.0%',
        )
    )
)